In [40]:
import os
import bz2

ROOT = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(""))), "machine-learning")
DATASET_P = os.path.join(ROOT, "datasets\\spam_classifier")
RAW_DATASET_P = os.path.join(DATASET_P, "raw")
EXTRACTED_DATASET_P = os.path.join(DATASET_P, "extracted")
PROCESSED_DATASET_P = os.path.join(DATASET_P, "processed")

def extract_data_from_bzf(source: str, destination: str, extension: str):
    if not extension.startswith("."):
        raise ValueError("extension must begin with a .")
    if os.listdir(destination) == []:
        print("Dataset has not been extracted")
        print("Extracting...")
        for f in os.listdir(source):
            decompressor = bz2.BZ2Decompressor()
            source_path = os.path.join(source, f)
            extraction_path = os.path.join(destination, f + extension)
            with open(extraction_path, 'wb') as bzfw, open(source_path, 'rb') as bzfr:
                for data in iter(lambda: bzfr.read(100*1024), b''):
                    bzfw.write(decompressor.decompress(data))
    else:
        print("Data already extracted")
        
extract_data_from_bzf(RAW_DATASET_P, EXTRACTED_DATASET_P, ".txt")

Data already extracted


In [41]:
def sanitize_data_from_extracted(source: str, destination: str):
    for file_name in os.listdir(source):
        read_path = source + "\\" + file_name
        write_path = destination + "\\" + file_name
        with open(read_path, 'r',encoding='latin1') as rf:
            with open(write_path, 'w', encoding='latin1') as wf:
                lines = rf.readlines()
                for line in lines:
                    processed_line = line.replace("\x00", "").replace("\n", "").replace("\t", "")
                    # processed_line = line.replace("\x00", "")
                    wf.write(processed_line)

sanitize_data_from_extracted(EXTRACTED_DATASET_P, PROCESSED_DATASET_P)


In [42]:
segment_markers = ["easy_ham/", "easy_ham2/", "hard_ham/", "spam/", "spam_2/"]

In [86]:
import re

pt = r"^(spam_2\/[\w\s\d\\.\/]+)(?=From)"
re.findall(pt, "spam_2/0000775000175000017500000000000010214426762011303 5ustar  jmjm00000000000000spam_2/00001.317e78fa8ee2f54cd4890fdc09ba81760000664000175000017500000001116107627640277016365 0ustar  jmjm00000000000000From")

['spam_2/0000775000175000017500000000000010214426762011303 5ustar  jmjm00000000000000spam_2/00001.317e78fa8ee2f54cd4890fdc09ba81760000664000175000017500000001116107627640277016365 0ustar  jmjm00000000000000']

In [82]:
re.findall(pt, "spam_2/0000775000175000017500000000000010214426762011303 5ustar  jmjm00000000000000spam_2/00001.317e78fa8ee2f54cd4890fdc09ba81760000664000175000017500000001116107627640277016365 0ustar  jmjm00000000000000From")
# re.fullmatch(pt, "spam_2/0000775000175000017500000000000010214426762011303 5ustar  jmjm00000000000000spam_2/00001.317e78fa8ee2f54cd4890fdc09ba81760000664000175000017500000001116107627640277016365 0ustar  jmjm00000000000000From")

['spam_2/0000775000175000017500000000000010214426762011303 5ustar  jmjm00000000000000spam_2/00001.317e78fa8ee2f54cd4890fdc09ba81760000664000175000017500000001116107627640277016365 0ustar  jmjm00000000000000']

In [83]:
import pandas as pd

def read_data_into_dataframe(source: str) -> pd.DataFrame:
    texts = []
    for file in os.listdir(source):
        read_path = source + "\\" + file
        with open(read_path, 'r', encoding='latin1') as f:
            raw = f.read()
            texts = re.split(pt, raw)
    return pd.DataFrame(texts)

In [84]:
df = read_data_into_dataframe(PROCESSED_DATASET_P)
df

,0
0,
1,spam_2/000077500017500001750000000000001021442...
2,From ilug-admin@linux.ie Tue Aug 6 11:51:02 ...


In [76]:
# preview
with open(DATASET_P + "preview.txt", 'w') as f:
    f.write(str(df.loc[2].values))